In [417]:
import pandas as pd

df_nirvana = pd.read_csv('Dados/nirvana_lyrics.csv')
df_pink_floyd = pd.read_csv('Dados\pink_floyd_lyrics.csv')

In [418]:
df_nirvana_treino = df_nirvana.iloc[:50]
df_pink_floyd_treino = df_pink_floyd.iloc[:50]

df_nirvana_teste = df_nirvana.iloc[50:100]
df_pink_floyd_teste = df_pink_floyd.iloc[50:100]

In [419]:
probabilidade_nirvana = len(df_nirvana_treino) / (len(df_nirvana_treino) + len(df_pink_floyd_treino))
probabilidade_pink_floyd = len(df_pink_floyd_treino) / (len(df_nirvana_treino) + len(df_pink_floyd_treino))

In [420]:
import re
def arruma(lyrics):
    if isinstance(lyrics, str):
        lyrics = lyrics.lower()
        lyrics = re.sub(r"[.!@#$%&*(){}:;_?/’<>+|=,\[\]\n']", '', lyrics)
        lyrics = re.sub(r'[\"]', '', lyrics)


    return lyrics

def split_function(texto):
    return texto.split()

df_nirvana_treino.loc[:,'lyrics'] = df_nirvana_treino['lyrics'].astype(str)
df_pink_floyd_treino.loc[:,'lyrics'] = df_pink_floyd_treino['lyrics'].astype(str)

df_nirvana_treino.loc[:,'lyrics'] = df_nirvana_treino['lyrics'].apply(arruma)
df_pink_floyd_treino.loc[:,'lyrics'] = df_pink_floyd_treino['lyrics'].apply(arruma)

df_nirvana_treino = pd.DataFrame(df_nirvana_treino)
df_pink_floyd_treino = pd.DataFrame(df_pink_floyd_treino)

def keep_unique_words(text):
    words = text.split()
    unique_words = set(words)
    return ' '.join(unique_words)

df_nirvana_treino['lyrics'] = df_nirvana_treino['lyrics'].apply(keep_unique_words)
df_pink_floyd_treino['lyrics'] = df_pink_floyd_treino['lyrics'].apply(keep_unique_words)


In [421]:
nirvaninha = {}
pinkinho = {}
contador1 = 0
contador2 = 0

for musicas_n in df_nirvana_treino['lyrics']:
    nirvaninha[contador1] = musicas_n.split()
    contador1 += 1

for musicas_pf in df_pink_floyd_treino['lyrics']:
    pinkinho[contador2] = musicas_pf.split()
    contador2 += 1

print(nirvaninha)
print(pinkinho)

{0: ['your', 'while', 'the', 'an', 'ill', 'me', 'in', 'see', 'no', 'lend', 'advantage', 'out', 'keep', 'im', 'to', 'easy', 'can', 'with', 'night', 'need', 'friend', 'every', 'pick', 'line', 'i', 'dry', 'hope', 'number', 'date', 'time', 'you', 'take', 'think', 'have', 'shoe', 'cant', 'wont', 'but', 'free', 'too', 'hang', 'a', 'fit', 'this', 'clue', 'do', 'standing', 'ear'], 1: ['reason', 'the', 'barely', 'could', 'an', 'what', 'we', 'be', 'and', 'where', 'idea', 'imitate', 'of', 'whats', 'heat', 'trend', 'facts', 'lack', 'theyll', 'its', 'everyday', 'all', 'fans', 'can', 'doesnt', 'sound', 'keep', 'love', 'ode', 'kids', 'matter', 'packaged', 'season', 'culture', 'friend', 'our', 'top', 'like', 'shit', 'upon', 'steal', 'world', 'not', 'will', 'up', 'eat', 'you', 'cant', 'have', 'moves', 'swear', 'if', 'equally', 'just', 'plan', 'yesterday', 'meaning', 'a', 'is', 'nowadays', 'crime', 'stage', 'forget', 'properly', 'lose', 'anyways', 'anyway', 'format', 'how', 'everything', 'it', 'anything

In [422]:
frequencia_nirvana = {}
frequencia_pink_floyd = {}

for musicas in nirvaninha.values():
    for palavras in musicas:
        if palavras in frequencia_nirvana:
            frequencia_nirvana[palavras] += 1
        else:
            frequencia_nirvana[palavras] = 1

for musicas in pinkinho.values():
    for palavras in musicas:
        if palavras in frequencia_pink_floyd:
            frequencia_pink_floyd[palavras] += 1
        else:
            frequencia_pink_floyd[palavras] = 1

print(frequencia_nirvana)
print(frequencia_pink_floyd)

{'your': 19, 'while': 2, 'the': 34, 'an': 7, 'ill': 4, 'me': 25, 'in': 29, 'see': 8, 'no': 10, 'lend': 2, 'advantage': 1, 'out': 10, 'keep': 2, 'im': 19, 'to': 35, 'easy': 1, 'can': 11, 'with': 14, 'night': 5, 'need': 7, 'friend': 4, 'every': 2, 'pick': 2, 'line': 2, 'i': 34, 'dry': 1, 'hope': 5, 'number': 1, 'date': 3, 'time': 7, 'you': 32, 'take': 11, 'think': 5, 'have': 19, 'shoe': 1, 'cant': 7, 'wont': 3, 'but': 9, 'free': 3, 'too': 2, 'hang': 2, 'a': 32, 'fit': 2, 'this': 10, 'clue': 1, 'do': 8, 'standing': 2, 'ear': 1, 'reason': 4, 'barely': 1, 'could': 8, 'what': 11, 'we': 11, 'be': 16, 'and': 26, 'where': 4, 'idea': 1, 'imitate': 1, 'of': 20, 'whats': 1, 'heat': 1, 'trend': 2, 'facts': 1, 'lack': 1, 'theyll': 2, 'its': 13, 'everyday': 1, 'all': 14, 'fans': 2, 'doesnt': 1, 'sound': 3, 'love': 7, 'ode': 1, 'kids': 1, 'matter': 1, 'packaged': 1, 'season': 3, 'culture': 1, 'our': 4, 'top': 1, 'like': 16, 'shit': 3, 'upon': 1, 'steal': 1, 'world': 1, 'not': 10, 'will': 5, 'up': 9, '

In [423]:
probabilidade_por_palavra_nirvana = {}
probabilidade_por_palavra_pink_floyd = {}

for palavra,quantidade in frequencia_nirvana.items():
    probabilidade_por_palavra_nirvana[palavra] = quantidade/len(nirvaninha)

for palavra,quantidade in frequencia_pink_floyd.items():
    probabilidade_por_palavra_pink_floyd[palavra] = quantidade/len(pinkinho)

print(probabilidade_por_palavra_nirvana)
print(probabilidade_por_palavra_pink_floyd)

{'your': 0.38, 'while': 0.04, 'the': 0.68, 'an': 0.14, 'ill': 0.08, 'me': 0.5, 'in': 0.58, 'see': 0.16, 'no': 0.2, 'lend': 0.04, 'advantage': 0.02, 'out': 0.2, 'keep': 0.04, 'im': 0.38, 'to': 0.7, 'easy': 0.02, 'can': 0.22, 'with': 0.28, 'night': 0.1, 'need': 0.14, 'friend': 0.08, 'every': 0.04, 'pick': 0.04, 'line': 0.04, 'i': 0.68, 'dry': 0.02, 'hope': 0.1, 'number': 0.02, 'date': 0.06, 'time': 0.14, 'you': 0.64, 'take': 0.22, 'think': 0.1, 'have': 0.38, 'shoe': 0.02, 'cant': 0.14, 'wont': 0.06, 'but': 0.18, 'free': 0.06, 'too': 0.04, 'hang': 0.04, 'a': 0.64, 'fit': 0.04, 'this': 0.2, 'clue': 0.02, 'do': 0.16, 'standing': 0.04, 'ear': 0.02, 'reason': 0.08, 'barely': 0.02, 'could': 0.16, 'what': 0.22, 'we': 0.22, 'be': 0.32, 'and': 0.52, 'where': 0.08, 'idea': 0.02, 'imitate': 0.02, 'of': 0.4, 'whats': 0.02, 'heat': 0.02, 'trend': 0.04, 'facts': 0.02, 'lack': 0.02, 'theyll': 0.04, 'its': 0.26, 'everyday': 0.02, 'all': 0.28, 'fans': 0.04, 'doesnt': 0.02, 'sound': 0.06, 'love': 0.14, 'o

In [424]:
import pandas as pd

def calculate_word_probability(word, df1, df2):
    def count_word_occurrences(dataframe, word):
        word_counts = 0

        for lyrics in dataframe:
            if pd.notna(lyrics):  
                word_count = lyrics.count(word.lower())  
                word_counts += word_count

        return word_counts

   
    count_df1 = count_word_occurrences(df1, word)
    count_df2 = count_word_occurrences(df2, word)

   
    total_count = count_df1 + count_df2
    if total_count == 0:
        return 0.0 
    probability_df1 = (count_df1 / total_count) * 100
    probability_df2 = (count_df2 / total_count) * 100

    return probability_df1, probability_df2


word = 'you'

prob_df1, prob_df2 = calculate_word_probability(word, df_pink_floyd_treino['lyrics'], df_nirvana_treino['lyrics'])
print(f'A probabilidade da palavra "{word}" pertencer ao Pink Floyd é de {prob_df1:.2f}%')
print(f'A probabilidade da palavra "{word}" pertencer ao Nirvana é de {prob_df2:.2f}%')

A probabilidade da palavra "you" pertencer ao Pink Floyd é de 50.71%
A probabilidade da palavra "you" pertencer ao Nirvana é de 49.29%
